In [1]:
using Pkg
Pkg.add(["DataFrames", "GLM", "Plots", "Random", "Lasso", "StatsBase", 
         "MLBase", "MultivariateStats", "HypothesisTests", "Distributions",
         "Bootstrap", "StatsPlots", "RDatasets", "MLJ", "MLJLinearModels"])


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed StatisticalTraits ──────── v3.4.0
   Installed CategoricalDistributions ─ v0.1.15
   Installed ContextVariablesX ──────── v0.1.3
   Installed Bootstrap ──────────────── v2.4.0
   Installed ShowCases ──────────────── v0.1.0
   Installed LearnAPI ───────────────── v0.1.0
   Installed NNlib ──────────────────── v0.9.30
   Installed DSP ────────────────────── v0.7.10
   Installed EnumX ──────────────────── v1.0.5
   Installed ScopedValues ───────────── v1.3.0
   Installed InitialValues ──────────── v0.3.1
   Installed MLUtils ────────────────── v0.4.8
   Installed PrettyPrint ────────────── v0.2.0
   Installed MLCore ─────────────────── v1.0.0
   Installed FiniteDiff ─────────────── v2.27.0
   Installed FLoopsBase ─────────────── v0.1.1
   Installed MLJFlow ────────────────── v0.5.0
   Installed EarlyStopping ──────────── v0.3.0
   Installed MLBase ─────────────────── v0.9.2
   Install

In [2]:
using DataFrames
using GLM
using Plots
using Random
using StatsBase
using Lasso
using MLBase
using MultivariateStats
using HypothesisTests
using Distributions
using Bootstrap
using StatsPlots
using RDatasets
using MLJ
using MLJLinearModels

In [3]:
# gen synthetic data for simple linear regression with controlled heteroscedasticity
n = 200
x = range(0, 10, length=n)
epsilon = randn(n) .* (0.5 .+ 0.3 .* x)  # Heteroscedastic errors
y = 2 .* x .+ 1 .+ epsilon


200-element Vector{Float64}:
  1.134477839865555
  0.8093936278375158
  0.38547332403073764
  1.3320269344781919
  1.50463986375581
  1.344415863751584
  1.3544913335960875
  1.5419626820460077
  2.1066407182293596
  1.7314719493006778
  ⋮
 15.13126217835801
 20.517039288361648
 19.92191627440723
 17.652433048726543
 23.627668351550476
 18.375399028105026
 22.548710009424894
 19.843284643184806
 16.02219167078583

In [4]:

#create a DataFrame
data = DataFrame(X=x, Y=y)

Row,X,Y
,Float64,Float64
1,0.0,1.13448
2,0.0502513,0.809394
3,0.100503,0.385473
4,0.150754,1.33203
5,0.201005,1.50464
6,0.251256,1.34442
7,0.301508,1.35449
8,0.351759,1.54196
9,0.40201,2.10664


In [5]:

#fit simple linear regression model
model_simple = lm(@formula(Y ~ X), data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Y ~ 1 + X

Coefficients:
───────────────────────────────────────────────────────────────────────
               Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────
(Intercept)  0.97552   0.322049    3.03    0.0028   0.340434    1.61061
X            1.94622   0.0557105  34.93    <1e-85   1.83636     2.05608
───────────────────────────────────────────────────────────────────────

In [6]:
println(model_simple)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Y ~ 1 + X

Coefficients:
───────────────────────────────────────────────────────────────────────
               Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────
(Intercept)  0.97552   0.322049    3.03    0.0028   0.340434    1.61061
X            1.94622   0.0557105  34.93    <1e-85   1.83636     2.05608
───────────────────────────────────────────────────────────────────────


In [7]:
coef_simple = coef(model_simple)
stderr_simple = stderror(model_simple)
tvalues_simple = coef_simple ./ stderr_simple
pvalues_simple = 2 * ccdf.(TDist(dof_residual(model_simple)), abs.(tvalues_simple))


2-element Vector{Float64}:
 0.002779706012531751
 1.3382569714512061e-86

In [8]:
coef_names = ["Intercept", "X"]
for i in 1:length(coef_simple)
    println("$(coef_names[i]): Estimate = $(round(coef_simple[i], digits=4)), " *
            "Std. Error = $(round(stderr_simple[i], digits=4)), " *
            "t-value = $(round(tvalues_simple[i], digits=4)), " *
            "p-value = $(round(pvalues_simple[i], digits=6))")
end


Intercept: Estimate = 0.9755, Std. Error = 0.322, t-value = 3.0291, p-value = 0.00278
X: Estimate = 1.9462, Std. Error = 0.0557, t-value = 34.9345, p-value = 0.0


In [10]:
predictions_simple = GLM.predict(model_simple)


200-element Vector{Float64}:
  0.9755198236319341
  1.0733197782015242
  1.1711197327711143
  1.2689196873407043
  1.3667196419102943
  1.4645195964798843
  1.562319551049474
  1.6601195056190643
  1.7579194601886543
  1.8557194147582443
  ⋮
 19.655311146423628
 19.753111100993216
 19.850911055562808
 19.948711010132396
 20.046510964701987
 20.144310919271575
 20.242110873841167
 20.339910828410755
 20.437710782980346

In [11]:
residuals_simple = residuals(model_simple)


200-element Vector{Float64}:
  0.15895801623362094
 -0.26392615036400846
 -0.7856464087403766
  0.06310724713748761
  0.13792022184551578
 -0.12010373272830033
 -0.20782821745338653
 -0.11815682357305657
  0.3487212580407053
 -0.12424746545756649
  ⋮
 -4.5240489680656175
  0.7639281873684318
  0.07100521884442301
 -2.2962779614058526
  3.5811573868484885
 -1.7689118911665496
  2.3065991355837276
 -0.49662618522594926
 -4.4155191121945165

In [12]:
sw_test = ShapiroWilkTest(residuals_simple)
println("Shapiro-Wilk test for normality: W = $(round(sw_test.W, digits=4)), p-value = $(round(pvalue(sw_test), digits=6))")
println("Normality assumption: $(pvalue(sw_test) > 0.05 ? "Not rejected" : "Rejected")")


Shapiro-Wilk test for normality: W = 0.9854, p-value = 0.036227
Normality assumption: Rejected


In [22]:
r_squared_simple = r2(model_simple)
adj_r_squared_simple = adjr2(model_simple)
rmse_simple = sqrt(mean(residuals_simple.^2))
mae_simple = mean(abs.(residuals_simple))


1.7150371348236353

In [23]:
println("R-squared: $(round(r_squared_simple, digits=4))")
println("Adjusted R-squared: $(round(adj_r_squared_simple, digits=4))")
println("RMSE: $(round(rmse_simple, digits=4))")
println("MAE: $(round(mae_simple, digits=4))")


R-squared: 0.8604
Adjusted R-squared: 0.8597
RMSE: 2.2743
MAE: 1.715


In [24]:
conf_int = confint(model_simple)
println("\nConfidence Intervals (95%):")
for i in 1:length(coef_names)
    println("$(coef_names[i]): ($(round(conf_int[i,1], digits=4)), $(round(conf_int[i,2], digits=4)))")
end



Confidence Intervals (95%):
Intercept: (0.3404, 1.6106)
X: (1.8364, 2.0561)


In [25]:
n = 300
x1 = range(0, 10, length=n)
x2 = 0.7 .* x1 .+ randn(n) .* 2
x3 = range(-5, 5, length=n)
x4 = randn(n) .* 3 
x5 = 0.3 .* x1 .+ 0.5 .* x3 .+ randn(n) 


300-element Vector{Float64}:
 -1.6071051138023362
 -2.5547293405084526
 -2.2890815019185475
 -3.6346052372579423
 -3.592817339240087
 -3.605561926130214
 -2.9840972887115766
 -2.495129037148038
 -1.3429286184698663
 -1.5229358887890676
  ⋮
  5.285947530374424
  4.285555773100454
  5.515477252640705
  4.374081615500912
  3.7488241945438183
  6.376301759512508
  6.044391749484387
  8.830929519833003
  6.356627101474468

In [26]:

true_beta = [2.0, 1.5, -0.5, 3.0, 0.0, 2.5]

#  response with some noise
y_multi = true_beta[1] .+ true_beta[2] .* x1 .+ true_beta[3] .* x2 .+ 
          true_beta[4] .* x3 .+ true_beta[5] .* x4 .+ true_beta[6] .* x5 .+ randn(n)

data_multi = DataFrame(Y=y_multi, X1=x1, X2=x2, X3=x3, X4=x4, X5=x5)

model_multi = lm(@formula(Y ~ X1 + X2 + X3 + X4 + X5), data_multi)


println(model_multi)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Y ~ 1 + X1 + X2 + X3 + X4 + X5

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                   Coef.   Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)   0.0         NaN          NaN       NaN     NaN          NaN
X1            1.87812       0.028111    66.81    <1e-99    1.82279      1.93344
X2           -0.479012      0.0276694  -17.31    <1e-46   -0.533466    -0.424557
X3            2.55261       0.0385141   66.28    <1e-99    2.47681      2.6284
X4            0.00322761    0.0182357    0.18    0.8596   -0.0326609    0.0391161
X5            2.56834       0.0571567   44.94    <1e-99    2.45585      2.68083
───────────────────────────────────────

In [27]:
coef_multi = coef(model_multi)
stderr_multi = stderror(model_multi)
tvalues_multi = coef_multi ./ stderr_multi
pvalues_multi = 2 * ccdf.(TDist(dof_residual(model_multi)), abs.(tvalues_multi))

println("\nCoefficient Statistics:")
coef_names_multi = ["Intercept", "X1", "X2", "X3", "X4", "X5"]
for i in 1:length(coef_multi)
    println("$(coef_names_multi[i]): Estimate = $(round(coef_multi[i], digits=4)), " *
            "Std. Error = $(round(stderr_multi[i], digits=4)), " *
            "t-value = $(round(tvalues_multi[i], digits=4)), " *
            "p-value = $(round(pvalues_multi[i], digits=6))")
end


Coefficient Statistics:
Intercept: Estimate = 0.0, Std. Error = NaN, t-value = NaN, p-value = NaN
X1: Estimate = 1.8781, Std. Error = 0.0281, t-value = 66.8107, p-value = 0.0
X2: Estimate = -0.479, Std. Error = 0.0277, t-value = -17.312, p-value = 0.0
X3: Estimate = 2.5526, Std. Error = 0.0385, t-value = 66.2772, p-value = 0.0
X4: Estimate = 0.0032, Std. Error = 0.0182, t-value = 0.177, p-value = 0.859634
X5: Estimate = 2.5683, Std. Error = 0.0572, t-value = 44.9351, p-value = 0.0


In [28]:
feature_importance = abs.(tvalues_multi[2:end])  # Exclude intercept
feature_names = coef_names_multi[2:end]
sorted_indices = sortperm(feature_importance, rev=true)

println("\nFeature Importance (based on absolute t-values):")
for i in sorted_indices
    println("$(feature_names[i]): $(round(feature_importance[i], digits=4))")
end


Feature Importance (based on absolute t-values):
X1: 66.8107
X3: 66.2772
X5: 44.9351
X2: 17.312
X4: 0.177


In [29]:
function calculate_vif(data, feature_names)
    vifs = Float64[]
    for i in 1:length(feature_names)
        target = feature_names[i]
        predictors = filter(x -> x != target, feature_names)
        if isempty(predictors)
            push!(vifs, 1.0)
            continue
        end
        formula = Term(Symbol(target)) ~ sum(Term.(Symbol.(predictors)))
        model_vif = lm(formula, data)
        r2_vif = r2(model_vif)
        push!(vifs, 1 / (1 - r2_vif))
    end
    return vifs
end

calculate_vif (generic function with 1 method)

In [30]:
vifs = calculate_vif(data_multi, ["X1", "X2", "X3", "X4", "X5"])


5-element Vector{Float64}:
 Inf
  2.0250965462543324
 Inf
  1.006695091681108
  7.311047742108615

In [31]:
println("\nVariance Inflation Factors (VIF):")
for i in 1:length(feature_names)
    println("$(feature_names[i]): $(round(vifs[i], digits=4)) $(vifs[i] > 5 ? "- High multicollinearity" : "")")
end


Variance Inflation Factors (VIF):
X1: Inf - High multicollinearity
X2: 2.0251 
X3: Inf - High multicollinearity
X4: 1.0067 
X5: 7.311 - High multicollinearity


In [33]:
function partial_correlation(data, x, y, controlling_vars)

    x_formula = Term(Symbol(x)) ~ sum(Term.(Symbol.(controlling_vars)))
    x_model = lm(x_formula, data)
    x_residuals = residuals(x_model)
    

    y_formula = Term(Symbol(y)) ~ sum(Term.(Symbol.(controlling_vars)))
    y_model = lm(y_formula, data)
    y_residuals = residuals(y_model)
    
    return cor(x_residuals, y_residuals)
end

partial_correlation (generic function with 1 method)

In [34]:
for i in 1:length(feature_names)
    controlling_vars = filter(x -> x != feature_names[i], feature_names)
    pc = partial_correlation(data_multi, feature_names[i], "Y", controlling_vars)
    println("$(feature_names[i]): $(round(pc, digits=4))")
end

X1: 0.0137
X2: -0.7099
X3: -0.0277
X4: 0.0103
X5: 0.9341


In [35]:
r_squared_multi = r2(model_multi)
adj_r_squared_multi = adjr2(model_multi)
residuals_multi = residuals(model_multi)
rmse_multi = sqrt(mean(residuals_multi.^2))
mae_multi = mean(abs.(residuals_multi))

println("\nPerformance Metrics:")
println("R-squared: $(round(r_squared_multi, digits=4))")
println("Adjusted R-squared: $(round(adj_r_squared_multi, digits=4))")
println("RMSE: $(round(rmse_multi, digits=4))")
println("MAE: $(round(mae_multi, digits=4))")


Performance Metrics:
R-squared: 0.9973
Adjusted R-squared: 0.9973
RMSE: 0.9462
MAE: 0.7762


In [37]:

function anova_test(model)
    n = nobs(model)
    p = length(coef(model)) - 1
    r2 = GLM.r2(model)
    f_stat = (r2 / p) / ((1 - r2) / (n - p - 1))
    p_val = 1 - cdf(FDist(p, n - p - 1), f_stat)
    return (f_statistic = f_stat, p_value = p_val, df1 = p, df2 = n - p - 1)
end

anova_result = anova_test(model_multi)
println("\nANOVA for Model Significance:")
println("F($(anova_result.df1), $(anova_result.df2)) = $(round(anova_result.f_statistic, digits=4)), p-value = $(round(anova_result.p_value, digits=6))")
println("Model significance: $(anova_result.p_value < 0.05 ? "Significant" : "Not significant")")


ANOVA for Model Significance:
F(5, 294.0) = 21973.4246, p-value = 0.0
Model significance: Significant


In [38]:
n_lasso = 200
p_lasso = 20  
X_lasso = randn(n_lasso, p_lasso)
beta_true_lasso = vcat([1.5, 0.8, 1.2, 0.6, 0.9], zeros(p_lasso-5)) 
y_lasso = X_lasso * beta_true_lasso + 0.5 * randn(n_lasso)


200-element Vector{Float64}:
  2.7549787019434926
 -1.9429753755391668
  2.9449346893522725
 -0.5372347200984396
  0.7062488494522423
 -2.3216423672899467
 -3.1647532081340537
 -1.8859403761433322
  2.6549929104976013
 -0.9765005022088569
  ⋮
  3.443101907383187
  0.25360941058690917
  0.31550078859947406
 -2.049248904594208
  3.982929369660077
  3.242496187754242
  1.4315538250661717
 -1.0607036343620784
 -0.7497444828885428

In [39]:
data_lasso = DataFrame(X_lasso, :auto)
data_lasso.Y = y_lasso

200-element Vector{Float64}:
  2.7549787019434926
 -1.9429753755391668
  2.9449346893522725
 -0.5372347200984396
  0.7062488494522423
 -2.3216423672899467
 -3.1647532081340537
 -1.8859403761433322
  2.6549929104976013
 -0.9765005022088569
  ⋮
  3.443101907383187
  0.25360941058690917
  0.31550078859947406
 -2.049248904594208
  3.982929369660077
  3.242496187754242
  1.4315538250661717
 -1.0607036343620784
 -0.7497444828885428

In [40]:
cv_folds = 10
println("Performing $cv_folds-fold cross-validation for lambda selection...")

X_matrix = Matrix(select(data_lasso, Not(:Y)))
y_vector = data_lasso.Y


Performing 10-fold cross-validation for lambda selection...


200-element Vector{Float64}:
  2.7549787019434926
 -1.9429753755391668
  2.9449346893522725
 -0.5372347200984396
  0.7062488494522423
 -2.3216423672899467
 -3.1647532081340537
 -1.8859403761433322
  2.6549929104976013
 -0.9765005022088569
  ⋮
  3.443101907383187
  0.25360941058690917
  0.31550078859947406
 -2.049248904594208
  3.982929369660077
  3.242496187754242
  1.4315538250661717
 -1.0607036343620784
 -0.7497444828885428

In [47]:
path = Lasso.fit(LassoPath, X_matrix, y_vector)

LassoPath (66) solutions for 21 predictors in 256 iterations):
───────────────────────────────────
               λ    pct_dev  ncoefs
───────────────────────────────────
 [1]  1.39921     0.0             0
 [2]  1.2749      0.0597632       1
 [3]  1.16164     0.10938         1
 [4]  1.05845     0.192506        2
 [5]  0.964418    0.267203        2
 [6]  0.878742    0.334948        3
 [7]  0.800677    0.409224        4
 [8]  0.729547    0.493796        4
 [9]  0.664736    0.564008        4
[10]  0.605683    0.622299        4
[11]  0.551875    0.670694        4
[12]  0.502848    0.710871        4
[13]  0.458177    0.749853        5
[14]  0.417473    0.785925        5
[15]  0.380386    0.815873        5
[16]  0.346594    0.840736        5
[17]  0.315803    0.861378        5
[18]  0.287748    0.878515        5
[19]  0.262186    0.892743        5
[20]  0.238894    0.904554        5
[21]  0.217671    0.914361        5
[22]  0.198334    0.922502        5
[23]  0.180714    0.929262        5
[